In [322]:
import os 
import sys 
import random 
import time

import numpy as np 
import pandas as pd 
import nltk

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
random.seed(16)  

from sklearn.metrics import precision_score, recall_score,  f1_score, accuracy_score
from sklearn import preprocessing

In [371]:
with open('train.tsv', 'r') as f: 
    data = f.readlines()
    
with open('test.txt', 'r') as f: 
    test_data = f.readlines()

In [372]:
for i in range(len(data)): 
    data[i] = data[i].split('\t')
    data[i][0] = data[i][0].lower()
    data[i][1] = data[i][1].strip('\n').lower().split('-')
    
for i in range(len(test_data)): 
    test_data[i] = test_data[i].strip('\n').lower()
    
splitted_test_data = [list(test_data[i]) for i in range(len(test_data))]

In [163]:
label_pool = []
char_pool = [] 
for i in range(len(data)):
    label_pool+= data[i][1]
    char_pool += list(data[i][0].lower())
label_pool = list(set(label_pool))
char_pool = list(set(char_pool))

In [164]:
set(label_pool).difference(set(char_pool))

{'an', 'ch', 'en', 'ng', 'on', 'ou', 'oun', 'ui'}

In [165]:
def create_labels(data_list):
    single_tag = 0 
    double_tag = 0 
    triple_tag = 0
    label_list = [] 
    for i in range(len(data_list)):
        temp_labels = []
        for j in range(len(data_list[i][1])): 
            if len(data_list[i][1][j])==1:
                temp_labels += ['B']
                single_tag += 1 
            elif len(data_list[i][1][j])==2:
                temp_labels += ['B','I']
                double_tag += 1 
            elif len(data_list[i][1][j])==3:
                temp_labels += ['B','I','I']
                triple_tag += 1 
        label_list.append(temp_labels)
    print("The number of single tags are: ", single_tag)
    print("The number of double tags are: ", double_tag)
    print("The number of triple tags are: ", triple_tag)
    return label_list

In [166]:
def split_random_data(input_data,labels, number=650):
    X_test = input_data.copy()
    y_test = labels.copy()
    random_list = sorted(random.sample(range(len(X_test)), number), reverse = True)
    X_train = []
    y_train = [] 
    for i in random_list:
        X_train.append(X_test[i])
        y_train.append(y_test[i])
        del X_test[i]
        del y_test[i]
    return X_train, X_test, y_train, y_test

In [167]:
labels = create_labels(data)

The number of single tags are:  72096
The number of double tags are:  7592
The number of triple tags are:  8


In [168]:
encoded_label_pool = []
for e in labels:
    encoded_label_pool+= e

encoded_label_pool = list(set(encoded_label_pool))

In [170]:
label_dict = {}
counter = 0 
for e in encoded_label_pool: 
    label_dict[e] = counter 
    counter += 1 
    
invers_label_dict = {v:k for k,v in label_dict.items()}

In [174]:
char_dict = {}
counter = 0 
for e in char_pool: 
    char_dict[e] = counter 
    counter += 1 

In [175]:
invers_char_dict = {v:k for k,v in char_dict.items()}
char_list = []

for e in data: 
    char_list.append(list(e[0]))

In [176]:
char_list[0]

['k', 'o', 'n', 's', 'i', 'l', 't', 'a', 'n']

In [180]:
X_train, X_test, y_train, y_test = split_random_data(char_list ,labels, number=650)

In [181]:
label_freq = [] 
for e in y_train:
    label_freq += e

In [182]:
fd = nltk.FreqDist(tag for tag in label_freq)
print(fd)

<FreqDist with 2 samples and 4367 outcomes>


In [183]:
X_y_zipped = [] 
X_list = []
y_list = [] 
for i in range(len(X_train)):
    X_y_zipped += list(zip(X_train[i],y_train[i]))
    X_list += X_train[i]
    y_list += y_train[i]

## HMM and EM
These algorithms and their implementations are mostly borrowed from the two following sources: 
1. https://github.com/kscanne/5755/blob/master/mutations/HMM.ipynb
2. https://github.com/kscanne/5755/blob/master/segment/WordAlignment.ipynb

The rest of the notebook is implemented without using any specific resources. 

In [184]:
cfd = nltk.ConditionalFreqDist(X_y_zipped)

In [185]:
cfd['g']

FreqDist({'B': 54, 'I': 3})

In [186]:
justwords = nltk.FreqDist(X_list).keys()
best_tags = dict((w, cfd[w].max()) for w in justwords)

In [187]:
print('Most frequent tag for "a":', best_tags['a'])
print('Most frequent tag for "g":', best_tags['g'])

Most frequent tag for "a": B
Most frequent tag for "g": B


In [188]:
Xy_train[0]

[('a', 'B'),
 ('v', 'B'),
 ('a', 'B'),
 ('n', 'I'),
 ('t', 'B'),
 ('o', 'B'),
 ('u', 'I')]

In [189]:
Xy_train = [] 
for i in range(len(X_train)):
    Xy_train.append(list(zip(X_train[i], y_train[i])))
    
Xy_test = [] 
for i in range(len(X_test)):
    Xy_test.append(list(zip(X_test[i], y_test[i])))

In [190]:
flipped = [(t,w) for sent in Xy_train for (w,t) in X_y_zipped]
wordgiventag = nltk.ConditionalFreqDist(flipped)


In [191]:
wordgiventag['I']

FreqDist({'n': 193700, 'u': 32500, 'h': 20150, 'g': 1950})

In [215]:
# this is P(w|t), unsmoothed!
def P(w,t):
    return wordgiventag[t][w] / wordgiventag[t].N()


print('P(a|B) =', P('a','B'))

P(a|B) = 0.12245922208281054


In [216]:
Xy_train[0]

[('d', 'B'),
 ('i', 'B'),
 ('s', 'B'),
 ('t', 'B'),
 ('r', 'B'),
 ('a', 'B'),
 ('k', 'B'),
 ('s', 'B'),
 ('y', 'B'),
 ('o', 'B'),
 ('n', 'I')]

In [237]:
tag_bigrams = [(x,y) for sent in Xy_train for x,y in nltk.bigrams([t for (w,t) in X_y_zipped])]


In [194]:
tag_bigram_counts = nltk.ConditionalFreqDist(tag_bigrams)
# this is count of noun tags following adjective tags (normal order in English)
print(tag_bigram_counts['B']['I'])
# this is count of adjective tags following noun tags
print(tag_bigram_counts['I']['I'])

248300
0


In [195]:
# this is P(t2|t1), unsmoothed again!
def tagP(t2,t1):
    return tag_bigram_counts[t1][t2] / tag_bigram_counts[t1].N()
print('P("I"|"B") =',tagP('I','B'))
print('P("B"|"B") =',tagP('B','B'))

P("I"|"B") = 0.09588353413654618
P("B"|"B") = 0.9041164658634538


In [196]:
y_train[0]

['B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 'I']

In [291]:
sentence_start = nltk.FreqDist(e[0] for e in y_train)
def initP(t):
    return sentence_start[t] / sentence_start.N()

In [292]:
def argmax(V,tag_list,t,i):
    ans=-1
    best=None
    for s in tag_list:
        temp=V[(s,i-1)]*tagP(t,s)
        if temp > ans:
            ans = temp
            best = s
    return (best,ans)

In [293]:
def printV(sentence,tag_list,V,B):
    for i in range(len(sentence)):
        print('i='+str(i)+' ['+sentence[i]+']')
        for t in tag_list:
            if V[(t,i)] != 0:
                toprint='  '+t+'='+str(V[(t,i)])
                if i>0:
                    toprint += ' (from '+B[(t,i)]+')'
                print(toprint)

In [294]:
test_dict = dict()
test_dict.copy()

{}

In [296]:
def get_sequence(sentence,tag_list,V,B):
    prob_dict = {}
    output_list = []
    for i in range(len(sentence)-1,-1,-1):
        max_val = -1
        for t in tag_list:
            if max_val < V[(t,i)]: 
                max_val = V[(t,i)]
                temp_tag = t 
        output_list.insert(0, temp_tag)
    return output_list

In [297]:
sentence_start.keys()

dict_keys(['B'])

In [305]:
def viterbi(sentence):
    V = dict()    # keys are (t,i) where t is a tag (row label) and i is position in sentence (column label)
    B = dict()    # same keys as V; this stores the "backpointers" to remember best tag sequence
    tag_list = label_dict.keys()
    for t in tag_list:
        V[(t,0)] = initP(t)*Probability[(sentence[0],t)]
    for i in range(1,len(sentence)):
        for t in tag_list:
            pair = argmax(V,tag_list,t,i)
            B[(t,i)] = pair[0]
            V[(t,i)] = pair[1]*Probability[(sentence[i],t)]
            
    
    return get_sequence(sentence,tag_list,V,B)

In [306]:
viterbi(list('touwon')) # T-ou-w-on

['B', 'B', 'I', 'B', 'B', 'I']

In [328]:
def P(w,t):
    return wordgiventag[t][w] / (wordgiventag[t].N()+np.finfo(float).eps)

Probability = {}
for w in char_dict.keys():
    for t in ['B', 'I']:
        Probability[(w,t)] = P(w,t)
        

In [329]:
Probability

{('k', 'B'): 0.047427854454203264,
 ('k', 'I'): 0.0,
 ('d', 'B'): 0.03136762860727729,
 ('d', 'I'): 0.0,
 ('s', 'B'): 0.06198243412797993,
 ('s', 'I'): 0.0,
 ('f', 'B'): 0.013801756587202008,
 ('f', 'I'): 0.0,
 ('m', 'B'): 0.0396486825595985,
 ('m', 'I'): 0.0,
 ('c', 'B'): 0.0077791718946047675,
 ('c', 'I'): 0.0,
 ('ò', 'B'): 0.01204516938519448,
 ('ò', 'I'): 0.0,
 ('r', 'B'): 0.04341279799247177,
 ('r', 'I'): 0.0,
 ('à', 'B'): 0.000752823086574655,
 ('à', 'I'): 0.0,
 ('b', 'B'): 0.01982434127979925,
 ('b', 'I'): 0.0,
 ('y', 'B'): 0.04015056461731493,
 ('y', 'I'): 0.0,
 ('o', 'B'): 0.0767879548306148,
 ('o', 'I'): 0.0,
 ('v', 'B'): 0.014554579673776663,
 ('v', 'I'): 0.0,
 ('u', 'B'): 0.0,
 ('u', 'I'): 0.13089005235602094,
 ('è', 'B'): 0.0328732747804266,
 ('è', 'I'): 0.0,
 ('l', 'B'): 0.0506900878293601,
 ('l', 'I'): 0.0,
 ('e', 'B'): 0.10388958594730238,
 ('e', 'I'): 0.0,
 ('w', 'B'): 0.017063989962358847,
 ('w', 'I'): 0.0,
 ('z', 'B'): 0.015307402760351318,
 ('z', 'I'): 0.0,
 ('i', '

In [331]:
Xy_test[0]

[('k', 'B'),
 ('o', 'B'),
 ('n', 'I'),
 ('s', 'B'),
 ('i', 'B'),
 ('l', 'B'),
 ('t', 'B'),
 ('a', 'B'),
 ('n', 'I')]

In [332]:
accuracy_list = []
counter = 0
total = 0

vit_output = [] 
for i in range(len(Xy_test)): 
    test_run = Xy_test[i]
    # list of tagged words
    # list of untagged words
    test_tagged_words = [w for (w,t) in Xy_test[i]]
    test_tagged_labels = [t for (w,t) in Xy_test[i]]
        
    output = viterbi(test_tagged_words)
    vit_output.append(output)
#         total += acc
    counter += 1 

    
# average = total/(counter + np.finfo(float).eps)

In [333]:
def compare_sequences(y_test, pred_test):
    total = 0 
    for i in range(len(y_test)):
        temp= 0 
        for j in range(len(y_test[i])):
#             print("y_test len: ", len(y_test[i]),"      pred_test" , len(pred_test[i]))
            if y_test[i][j] == pred_test[i][j]:
                temp += 1 
        total += temp/ len(y_test[i])
    return total/len(y_test)

In [334]:
compare_sequences(y_test,vit_output)

0.9780201107261873

In [335]:
# one iteration of EM
def EM(P, sc, tc, sv, tv):
    C = dict()   # keys are pairs (s,t); count of number of alignments
    for s in sv:
        for t in tv:
            C[(s,t)]=0
    total = dict()  # keys are source language words
    for t in tv:
        total[t]=0
    for ssent, tsent in zip(sc,tc):
        sent_totals=dict()
        for s in ssent:
            sent_totals[s]=0
            for t in tsent:
                sent_totals[s] += P[(s,t)]
        for s in ssent:
            for t in tsent:
                C[(s,t)] += P[(s,t)]/sent_totals[s]
                total[t] += P[(s,t)]/sent_totals[s]
    for t in tv:
        for s in sv:
            P[(s,t)] = C[(s,t)]/total[t]

In [336]:
for i in range(1000):
    EM(Probability,X_test,y_test,char_dict.keys(),label_dict.keys())

In [379]:
accuracy_list = []
counter = 0
total = 0

vit_output = [] 
for i in range(len(Xy_test)): 
    test_run = Xy_test[i]
    # list of tagged words
    # list of untagged words
    test_tagged_words = [w for (w,t) in Xy_test[i]]
    test_tagged_labels = [t for (w,t) in Xy_test[i]]
    
   
    output = viterbi(test_tagged_words)
    vit_output.append(output)
#         total += acc
    counter += 1 

    
compare_sequences(y_test,vit_output)

0.9780201107261873

In [382]:
le = preprocessing.LabelEncoder()
le.fit(y_list)
t_labels = le.transform([e for l in y_test[:100] for e in l])
p_labels = le.transform([e for l in vit_output[:100] for e in l])

f1_score(t_labels, p_labels)

0.9203539823008848

In [404]:
print("The f1 score based on \"I\" tags is: %.5f"%(f1_score(t_labels, p_labels, pos_label=1)))
print("The f1 score based on \"B\" tags is: %.5f"%(f1_score(t_labels, p_labels, pos_label=0)))

The f1 score based on "I" tags is: 0.92035
The f1 score based on "B" tags is: 0.99210


In [338]:
Probability

{('k', 'B'): 0.048633902882078237,
 ('k', 'I'): 0.0,
 ('d', 'B'): 0.03165816659153123,
 ('d', 'I'): 0.0,
 ('s', 'B'): 0.06139206493894808,
 ('s', 'I'): 0.0,
 ('f', 'B'): 0.017502933069755606,
 ('f', 'I'): 0.0,
 ('m', 'B'): 0.035572602677161454,
 ('m', 'I'): 0.0,
 ('c', 'B'): 0.0077629725738601295,
 ('c', 'I'): 0.0,
 ('ò', 'B'): 0.011901467290654865,
 ('ò', 'I'): 0.0,
 ('r', 'B'): 0.04535210293149933,
 ('r', 'I'): 0.0,
 ('à', 'B'): 0.0002108787116838056,
 ('à', 'I'): 0.0,
 ('b', 'B'): 0.021654607706030657,
 ('b', 'I'): 0.0,
 ('y', 'B'): 0.03785272874724236,
 ('y', 'I'): 0.0,
 ('o', 'B'): 0.07218641899326382,
 ('o', 'I'): 0.0,
 ('v', 'B'): 0.01605314192692934,
 ('v', 'I'): 0.0,
 ('u', 'B'): 0.0,
 ('u', 'I'): 0.1313702850772633,
 ('è', 'B'): 0.03312113765383799,
 ('è', 'I'): 0.0,
 ('l', 'B'): 0.05372135180144822,
 ('l', 'I'): 0.0,
 ('e', 'B'): 0.10219709564976791,
 ('e', 'I'): 0.0,
 ('w', 'B'): 0.017041635887947423,
 ('w', 'I'): 0.0,
 ('z', 'B'): 0.016501259189257506,
 ('z', 'I'): 0.0,
 (

In [363]:
def decode(X, y): 
    output = [] 
    for i in range(len(X)):
        temp_str = ''
        temp_output = []
        for j in range(len(X[i])):
            if y[i][j] == 'B':
                temp_str += X[i][j]
                temp_output.append(temp_str)
                temp_str = ''
            elif y[i][j] == 'I':
                if len(temp_output)<1: 
                    temp_output.append(X[i][j])
                else:
                    temp_output[-1] += X[i][j]
            
        output.append('-'.join(temp_output))
        
    return output 

In [364]:
output = decode(X_test, vit_output)

output[:3]

In [399]:
accuracy_list = []
counter = 0
total = 0

vit_output = [] 
for i in range(len(splitted_test_data)): 
    test_tagged_words = [w for w in splitted_test_data[i]]
    output = viterbi(test_tagged_words)
    vit_output.append(output)

output = decode(splitted_test_data, vit_output)

In [401]:
with open("semi_supervised.tsv", 'w') as f: 
    pass
for i in range(len(output)): 
    with open("semi_supervised.tsv", 'a+') as f: 
        f.write(test_data[i]+'\t'+output[i]+'\n')